# Análisis y Modelado de Series de Tiempo de Combustibles en Guatemala: Consumo, Importación y Precios (2000–2025)

## Carga, limpieza y preparación de datos

### Librerias

In [114]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

# import pmdarima as pm  # auto_arima

from prophet import Prophet

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPRegressor  # NN simple o usar tensorflow dependerá por ahora el simple

### Variables - Constantes

In [115]:
DATA_DIR = "./data-input"

# Consumo (ventas)
CONSUMO_2024_XLSX = f"{DATA_DIR}/CONSUMO-HIDROCARBUROS-2024-12.xlsx"
CONSUMO_2025_XLSX = f"{DATA_DIR}/VENTAS-HIDROCARBUROS-2025-05.xlsx"

# Importaciones
IMPORT_2024_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx"
IMPORT_2025_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx"

# Precios diarios
PRECIOS_2025_XLSX = f"{DATA_DIR}/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx"

OUTPUT_DIR = "./data-output"
CONSUMO_CSV = f"{OUTPUT_DIR}/consumo_combustibles.csv"
IMPORT_CSV = f"{OUTPUT_DIR}/importacion_combustibles.csv"
PRECIOS_CSV = f"{OUTPUT_DIR}/precios_diarios.csv"

SAVE_CONSUMO_CSV = True
SAVE_IMPORT_CSV = True
SAVE_PRECIOS_CSV = True
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [116]:
# df_consumo_raw_2024       # DataFrame original de consumo 2024
# df_consumo_raw_2025       # DataFrame original de consumo 2025
# df_consumo                # Unión y limpieza final de consumo mensual

# df_import_raw_2024        # Importaciones 2024
# df_import_raw_2025        # Importaciones 2025
# df_import                 # Unión y limpieza final de importación mensual
# df_import['diesel_total'] # Diésel combinado (alto + bajo azufre)

# df_precios_raw            # Precios diarios
# df_precios                # Promedio mensual gasolina regular


# serie_precio_gasolina_regular      # Series de tiempo de precios mensual (Ciudad Capital)
# serie_import_diesel_total          # Serie mensual de importación total de diésel
# serie_consumo_gas_licuado          # Serie mensual de consumo de gas propano


### Carga y conversión de archivos Excel a CSV

In [117]:
def sanitizeDateColumn(df, dateColumn="Fecha", convert=False, dropInvalid=False):
    """
    Si convert=True, intenta convertir a datetime.
    Si dropInvalid=True, elimina las fechas inválidas (NaT).
    """
    df = df.copy()
    
    if convert:
        df[dateColumn] = pd.to_datetime(df[dateColumn], errors="coerce")

        if dropInvalid:
            df = df.dropna(subset=[dateColumn])
    
    return df

In [118]:
def loadAndProcessConsumption(saveCsv=True):
    df_2024 = pd.read_excel(CONSUMO_2024_XLSX, skiprows=6)
    df_2025 = pd.read_excel(CONSUMO_2025_XLSX, skiprows=6)

    df_2024.columns = df_2024.columns.str.strip()
    df_2025.columns = df_2025.columns.str.strip()

    rename_map = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Diesel alto azufre": "dieselHS",
        "Gas licuado de petróleo": "glp"
    }

    df_2024 = df_2024.rename(columns=rename_map)
    df_2025 = df_2025.rename(columns=rename_map)

    df_2024["diesel_total"] = df_2024.get("dieselHS", 0)
    df_2025["diesel_total"] = df_2025.get("dieselLS", 0)

    df = pd.concat([df_2024, df_2025], ignore_index=True)
    df = df[["Fecha", "regular", "super", "glp", "diesel_total"]]

    # Forzar conversión a datetime y eliminar errores
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df.dropna(subset=["Fecha"])

    df = df.sort_values("Fecha")

    if saveCsv:
        df.to_csv(IMPORT_CSV, index=False)

    return df


In [119]:
def loadAndProcessImportation(saveCsv=True):
    """Carga y unifica los archivos de importación. Crea diesel_total sumando HS y LS según año."""
    df_2024 = pd.read_excel(IMPORT_2024_XLSX, skiprows=6)
    df_2025 = pd.read_excel(IMPORT_2025_XLSX, skiprows=6)

    # Limpia nombres de columnas
    df_2024.columns = df_2024.columns.str.strip()
    df_2025.columns = df_2025.columns.str.strip()

    rename_map_2024 = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Diesel alto azufre": "dieselHS",
        "Gas licuado de petróleo": "glp"
    }

    rename_map_2025 = {
        "Gasolina regular": "regular",
        "Gasolina superior": "super",
        "Diesel bajo azufre": "dieselLS",
        "Gas Licuado de Petróleo": "glp"
    }

    df_2024 = df_2024.rename(columns=rename_map_2024)
    df_2025 = df_2025.rename(columns=rename_map_2025)

    # Construir diesel_total por año
    df_2024["diesel_total"] = df_2024.get("dieselHS", 0)
    df_2025["diesel_total"] = df_2025.get("dieselLS", 0)

    df = pd.concat([df_2024, df_2025], ignore_index=True)
    df = df[["Fecha", "regular", "super", "glp", "diesel_total"]]

    # Sanitiza fechas
    df = sanitizeDateColumn(df, "Fecha", convert=True, dropInvalid=True)
    df = df.sort_values("Fecha")

    if saveCsv:
        df.to_csv(IMPORT_CSV, index=False)

    return df

In [120]:
def loadAndProcessPrices(saveCsv=True):
    """Carga precios diarios, renombra columnas clave, limpia fechas y genera promedios mensuales."""
    df = pd.read_excel(PRECIOS_2025_XLSX, skiprows=7)  # Omite encabezado doble

    # Limpieza de nombres de columnas
    df.columns = df.columns.str.strip()

    # Reasigna nombres de columnas manualmente (posición basada en estructura conocida)
    df = df.rename(columns={
        df.columns[0]: "Fecha",
        df.columns[2]: "super",
        df.columns[3]: "regular",
        df.columns[4]: "diesel",
        df.columns[6]: "glp"
    })

    # Filtramos solo columnas relevantes
    df = df[["Fecha", "regular", "super", "diesel", "glp"]]

    # Ahora sí: convertir fechas y eliminar inválidas
    df = sanitizeDateColumn(df, "Fecha", convert=True, dropInvalid=True)

    # Ordenar cronológicamente
    df = df.sort_values("Fecha")

    # Agregación mensual por promedio
    df_mensual = df.set_index("Fecha").resample("M").mean().reset_index()

    if saveCsv:
        df_mensual.to_csv(PRECIOS_CSV, index=False)

    return df_mensual

In [121]:
df_consumo = loadAndProcessConsumption(saveCsv=SAVE_CONSUMO_CSV)
df_importacion = loadAndProcessImportation(saveCsv=SAVE_IMPORT_CSV)
df_precios = loadAndProcessPrices(saveCsv=SAVE_PRECIOS_CSV)


C:\Users\josue\AppData\Local\Temp\ipykernel_25704\3229522666.py:27: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_mensual = df.set_index("Fecha").resample("M").mean().reset_index()


In [122]:
# # Re-guardar por si SAVE_* son False
# df_consumo.to_csv(CONSUMO_CSV, index=False)
# df_importacion.to_csv(IMPORT_CSV, index=False)
# df_precios.to_csv(PRECIOS_CSV, index=False)

# # Verificación de forma (shape)
# print("✔ Data saved and loaded:")
# print(f"Consumo: {df_consumo.shape}")
# print(f"Importación: {df_importacion.shape}")
# print(f"Precios: {df_precios.shape}")
